In [1]:
# -*- coding: utf-8 -*-
import json
import os
root = r"D:\data\数据资料\biendata\covid19赛道二关系抽取\task2_public"
train_text = "task2_user_train.json"
train_label = "task2_train_label.json"

def openfile(file):
    with open(file) as f:
        return json.load(f)

train_texts = openfile(os.path.join(root, train_text))
train_labels = openfile(os.path.join(root, train_label))

print(train_texts[0])
print(train_labels[0])

{'train_1': 'Safety and efficacy of intravenous bimagrumab in inclusion body myositis (RESILIENT): a randomised, double-blind, placebo-controlled phase 2b trial\tBimagrumab showed a good safety profile, relative to placebo, in individuals with inclusion body myositis but did not improve 6MWD. The strengths of our study are that, to the best of our knowledge, it is the largest randomised controlled trial done in people with inclusion body myositis, and it provides important natural history data over 12 months.'}
{'train_1': [{'head': {'word': 'Bimagrumab', 'start': 148, 'end': 158}, 'rel': 'PositivelyRegulates', 'tail': {'word': 'inclusion body myositis', 'start': 230, 'end': 253}}]}


In [2]:
print(len(train_texts))
print(len(train_labels))

1326
1326


In [1]:
# 因为用到了transformer的tokenizer， 需要结合分词接过来修改标注开始的地方。
# import transformers

from transformers import AutoTokenizer
pretrain_model_weight = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
test_sentence = "Safety and efficacy of intravenous bimagrumab in inclusion body myositis (RESILIENT): a randomised, double-blind, placebo-controlled phase 2b trial\tBimagrumab showed a good safety profile, relative to placebo, in individuals with inclusion body myositis but did not improve 6MWD. The strengths of our study are that, to the best of our knowledge, it is the largest randomised controlled trial done in people with inclusion body myositis, and it provides important natural history data over 12 months."

In [2]:
tokenizer.vocab.get("Here")

3446

In [5]:
test_sentence_2 = "Safety and efficacy of intravenous bimagrumab in inclusion body myositis (RESILIENT): a randomised"
tokend = tokenizer(test_sentence_2, add_special_tokens=False)
tokend

{'input_ids': [9218, 1105, 23891, 1104, 1107, 4487, 7912, 2285, 16516, 1918, 1403, 5697, 6639, 1107, 10838, 1404, 1139, 2155, 10721, 113, 155, 9919, 17656, 17444, 15681, 114, 131, 170, 7091, 3673], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
import torch
batch_size = 6
bert_emb_size = 765
hidden_size = 512
inp = torch.randn(batch_size, bert_emb_size)
print(inp.size())
hid = torch.randn(batch_size, hidden_size)
print(hid.size())

torch.cat([inp, hid], dim=1).size()

torch.Size([6, 765])
torch.Size([6, 512])


torch.Size([6, 1277])

In [10]:
from transformers import AutoModel
import torch
model =  AutoModel.from_pretrained(pretrain_model_weight)

In [26]:
input_ids_tensor = torch.tensor(tokend.get('input_ids'), dtype=torch.long).unsqueeze(dim=0)
# print(input_ids_tensor.size())
atten_mask_tensor = torch.tensor(tokend.get('attention_mask'), dtype=torch.long).unsqueeze(dim=0)

inputs = {"input_ids": input_ids_tensor,
          "attention_mask": atten_mask_tensor}
outputs = model(**inputs)
# print(outputs.loss.size())
print(outputs[0].size())
print(outputs[1].size())

torch.Size([1, 30, 768])
torch.Size([1, 768])


In [4]:
def get_t_char_and_len(t):
    if t.startswith("##"):
        return t[2:], len(t)-2
    else:
        return t, len(t)

def chars_token_align(sentence, tokenized):
    """逐个对齐，"""
    s_idx = 0
    token_s_e_idx = []
    for idx, t in enumerate(tokenized):
        t_char, t_len = get_t_char_and_len(t)
        
        token_s_idx = sentence.find(t_char, s_idx, s_idx+t_len+3)
        if token_s_idx != -1:
            token_e_idx = token_s_idx + t_len
        else:
            print(f"error at {idx}: {t}")
            break 
        token_s_e_idx.append((token_s_idx, token_e_idx))

        s_idx = token_e_idx
    return  token_s_e_idx

In [5]:
test_tokened = tokenizer.tokenize(test_sentence)
test_tokened

['Safety',
 'and',
 'efficacy',
 'of',
 'in',
 '##tra',
 '##ven',
 '##ous',
 'bi',
 '##ma',
 '##g',
 '##rum',
 '##ab',
 'in',
 'inclusion',
 'body',
 'my',
 '##os',
 '##itis',
 '(',
 'R',
 '##ES',
 '##IL',
 '##IE',
 '##NT',
 ')',
 ':',
 'a',
 'random',
 '##ised',
 ',',
 'double',
 '-',
 'blind',
 ',',
 'place',
 '##bo',
 '-',
 'controlled',
 'phase',
 '2',
 '##b',
 'trial',
 'B',
 '##ima',
 '##g',
 '##rum',
 '##ab',
 'showed',
 'a',
 'good',
 'safety',
 'profile',
 ',',
 'relative',
 'to',
 'place',
 '##bo',
 ',',
 'in',
 'individuals',
 'with',
 'inclusion',
 'body',
 'my',
 '##os',
 '##itis',
 'but',
 'did',
 'not',
 'improve',
 '6',
 '##M',
 '##WD',
 '.',
 'The',
 'strengths',
 'of',
 'our',
 'study',
 'are',
 'that',
 ',',
 'to',
 'the',
 'best',
 'of',
 'our',
 'knowledge',
 ',',
 'it',
 'is',
 'the',
 'largest',
 'random',
 '##ised',
 'controlled',
 'trial',
 'done',
 'in',
 'people',
 'with',
 'inclusion',
 'body',
 'my',
 '##os',
 '##itis',
 ',',
 'and',
 'it',
 'provides',
 'i

In [6]:
class Token_stru:
    def __init__(self, string):
        self.string = string
        self.string_start = None
        self.string_end = None 
        self.token_idx = None 
        self.token_tag = None 
    
    def __repr__(self):
        return f"{self.token_idx}\t{self.string}\t{self.string_start}\t{self.string_end}\t{self.token_tag}"

In [7]:
test_tokened_stru = [Token_stru(i) for i in test_tokened]
 

In [8]:
test_token_s_e_idx = chars_token_align(test_sentence, test_tokened)

In [9]:
for idx, (stru, s_e_idx) in enumerate(zip(test_tokened_stru, test_token_s_e_idx)):
    stru.string_start = s_e_idx[0]
    stru.string_end = s_e_idx[1]
    stru.token_idx = idx 

In [10]:
print(test_tokened_stru[0].string)
print(test_tokened_stru[0].string_start)
print(test_tokened_stru[0].string_end)
print(test_tokened_stru[0].token_idx)

Safety
0
6
0


In [11]:
train_labels[0].get("train_1")

[{'head': {'word': 'Bimagrumab', 'start': 148, 'end': 158},
  'rel': 'PositivelyRegulates',
  'tail': {'word': 'inclusion body myositis', 'start': 230, 'end': 253}}]

In [12]:
def tag_head(labels, tokened_struc):
    for label in labels:
        head_word = label.get("head").get("word")
        head_start = label.get("head").get("start")
        head_end = label.get("head").get("end")

        for j in tokened_struc:
            if head_start == j.string_start:
                j.token_tag = "B-head"
            if head_end == j.string_end:
                j.token_tag = "E-head"
            if head_start == head_end:
                j.token_tag = "U-head"
    return tokened_struc

     

In [13]:
test_labels = train_labels[0].get("train_1")
test_tokened_stru = tag_head(test_labels, test_tokened_stru)


In [14]:
print(len(test_tokened_stru))

119


In [15]:
with open("test_draft.txt", "w", encoding="utf-8", newline="") as f:
    for i in test_tokened_stru:
        tag = i.token_tag 
        tag = "O" if tag is None else tag
        f.write(f"{i.token_idx}\t{i.string}\t{i.string_start}\t{i.string_end}\t{tag}\n")


In [5]:
import torch
import torch.nn as nn
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=1)
src = torch.rand(10, 32, 512)
out  = transformer_encoder(src)
out.size()

torch.Size([10, 32, 512])

In [4]:
import os
a = os.listdir("./data/head") 
lenghts = []
for i in a:
    with open(os.path.join(r"./data/head", i), encoding="utf-8") as f:
        lenghts.append(len(f.readlines()))

import pandas as pd
b = pd.Series(lenghts)
              

In [5]:
b.describe()

count    1269.000000
mean      199.212766
std       156.645782
min        34.000000
25%       100.000000
50%       137.000000
75%       234.000000
max      1169.000000
dtype: float64

In [7]:
h = 0
for i in range(10):
    print(h)
    h = h+1
    print(h)
    print("="*20)

0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10


In [10]:
import torch 
batch_size = 6
hidden_size = 7
feature_size = 9
batch1 = torch.randn(batch_size, hidden_size)
batch2 = torch.randn(batch_size, feature_size)

torch.cat([batch1, batch2],dim=1).size()

torch.Size([6, 16])

In [11]:
batch_size = 6
seq_len = 12
num_tags = 5
pred = torch.randn(batch_size, seq_len, num_tags)
truth = torch.randn(batch_size, seq_len)

a = pred.view(-1, num_tags)
b = truth.view(-1)

print(a.size())
print(b.size())

torch.Size([72, 5])
torch.Size([72])


In [5]:
# torch conv1d用法
import torch 
import torch.nn as nn

batch_size = 16
seq_len = 512
feature_dim = 200
tag_num = 10

input = torch.randn(batch_size,seq_len,feature_dim)
m = nn.Conv1d(feature_dim,tag_num, 1)
output = m(input.permute(0,2,1))
print(output.size())

torch.Size([16, 10, 512])


In [7]:
# torch 切片【】
a = torch.randn(4,10,5)
b = a[:, 1:4, :]
print(b.size())

torch.Size([4, 3, 5])
